In [2]:
import os
import dotenv
from scholarly import scholarly
from langchain_community.tools.google_scholar import GoogleScholarQueryRun
from langchain_community.utilities.google_scholar import GoogleScholarAPIWrapper

In [6]:
# serp api
dotenv_file = dotenv.find_dotenv("/home/sangbin_yun/dev/llm_agent/.env")
dotenv.load_dotenv(dotenv_file)
# os.environ["SERP_API_KEY"]
tool = GoogleScholarQueryRun(api_wrapper=GoogleScholarAPIWrapper())
tool.run("hippocampus")

'Title: The hippocampus and behavior.\nAuthors: \nSummary: RJ Douglas - Psychological bulletin, 1967 - psycnet.apa.org\nTotal-Citations: 1591\n\nTitle: What does the hippocampus really do?\nAuthors: \nSummary: LE Jarrard - Behavioural brain research, 1995 - Elsevier\nTotal-Citations: 345\n\nTitle: The hippocampus book\nAuthors: \nSummary: P Andersen - 2007 - books.google.com\nTotal-Citations: 1892\n\nTitle: The hippocampus\nAuthors: JJ Knierim\nSummary: JJ Knierim - Current Biology, 2015 - cell.com\nTotal-Citations: 524\n\nTitle: Hormones and the hippocampus\nAuthors: R Lathe\nSummary: R Lathe - Journal of Endocrinology, 2001 - researchgate.net\nTotal-Citations: 381\n\nTitle: Hippocampus, space, and memory\nAuthors: JT Becker\nSummary: DS Olton, JT Becker, GE Handelmann - Behavioral and Brain …, 1979 - cambridge.org\nTotal-Citations: 2486\n\nTitle: Hippocampus\nAuthors: \nSummary: M Witter - The mouse nervous system, 2012 - Elsevier\nTotal-Citations: 62\n\nTitle: Hippocampus, time, and

In [9]:
search_query = scholarly.search_pubs('"conflict" AND EEG')

In [10]:
search_query.total_results

599000

In [12]:
scholarly.pprint(next(search_query))

b"{'author_id': ['OcKWjEwAAAAJ', 'PDZYdSkAAAAJ'],\n 'bib': {'abstract': 'would also be modulated by conflict. We applied linear  '\n                     'EEG response conflict tasks, we  also replicated '\n                     'conflict effects previously observed at the scalp-level '\n                     '(eg, midfrontal theta conflict',\n         'author': ['MX Cohen', 'KR Ridderinkhof'],\n         'pub_year': '2013',\n         'title': 'EEG source reconstruction reveals frontal-parietal dynamics '\n                  'of spatial conflict processing',\n         'venue': 'PloS one'},\n 'citedby_url': '/scholar?cites=13463466639366865087&as_sdt=5,33&sciodt=0,33&hl=en',\n 'eprint_url': 'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0057293',\n 'filled': False,\n 'gsrank': 2,\n 'num_citations': 137,\n 'pub_url': 'https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0057293',\n 'source': 'PUBLICATION_SEARCH_SNIPPET',\n 'url_add_sclib': '/citations?hl=en&xs